In [4]:
import os
import pytsk3

# Define the disk image path
image_path = "path_to_disk_image.dd"  # Replace with your disk image file

# Output directory for recovered files
output_dir = "recovered_files"
os.makedirs(output_dir, exist_ok=True)

def recover_deleted_files(partition_addr, output_dir):
    img_info = pytsk3.Img_Info(image_path)
    fs_info = pytsk3.FS_Info(img_info, addr=partition_addr)
    
    for directory_entry in fs_info.open_dir(path="/", addr=partition_addr):
        for file_entry in directory_entry:
            if file_entry.info.name.type == pytsk3.TSK_FS_NAME_TYPE_UNDEF:
                continue
            try:
                file_content = fs_info.open_meta(file_entry.info)
                if file_content.info.meta.flags == pytsk3.TSK_FS_META_FLAG_ALLOC:
                    with open(os.path.join(output_dir, file_entry.info.name.name), "wb") as f:
                        f.write(file_content.read_random(0, file_entry.info.meta.size))
            except Exception as e:
                print(f"Error recovering {file_entry.info.name.name}: {str(e)}")

if __name__ == "__main__":
    partition_table = pytsk3.Volume_Info(image_path)
    for partition in partition_table:
        if len(partition) > 0:
            print(f"Recovering deleted files from partition {partition.addr}")
            recover_deleted_files(partition.addr, output_dir)

print("File recovery complete. Recovered files are in the 'recovered_files' directory.")


ModuleNotFoundError: No module named 'pytsk3'